# Building Multi-class image classification models using very little data

## On the relevance of deep learning for small-data problems

**A message that I hear often is that "deep learning is only relevant when you have a huge amount of data". While not entirely incorrect, this is somewhat misleading. Certainly, deep learning requires the ability to learn features automatically from the data, which is generally only possible when lots of training data is available --especially for problems where the input samples are very high-dimensional, like images. However, convolutional neural networks --a pillar algorithm of deep learning-- are by design one of the best models available for most "perceptual" problems (such as image classification), even with very little data to learn from. Training a convnet from scratch on a small image dataset will still yield reasonable results, without the need for any custom feature engineering. Convnets are just plain good. They are the right tool for the job.**

**But what's more, deep learning models are by nature highly repurposable: you can take, say, an image classification or speech-to-text model trained on a large-scale dataset then reuse it on a significantly different problem with only minor changes, as we will see in this post. Specifically in the case of computer vision, many pre-trained models (usually trained on the ImageNet dataset) are now publicly available for download and can be used to bootstrap powerful vision models out of very little data.**

## 1. Data Augmentation

In this tutorial, we will present a few simple yet effective methods that you can use to build a powerful image classifier, using only very few training examples , just a few hundred or thousand pictures from each class you want to be able to recognize.

In order to make the most of our few training examples, we will "augment" them via a number of random transformations, so that our model would never see twice the exact same picture. This helps prevent overfitting and helps the model generalize better.

**Our setup: only 2400 training examples (800 per class)**

### Q. Why Data Augmentation ?

The right tool for an image classification job is a convnet, so let's try to train one on our data, as an initial baseline. Since we only have few examples, our number one concern should be overfitting. Overfitting happens when a model exposed to too few examples learns patterns that do not generalize to new data, i.e. when the model starts using irrelevant features for making predictions. For instance, if you, as a human, only see three images of people who are lumberjacks, and three, images of people who are sailors, and among them only one lumberjack wears a cap, you might start thinking that wearing a cap is a sign of being a lumberjack as opposed to a sailor. You would then make a pretty lousy lumberjack/sailor classifier.

In [1]:
!pip install Augmentor

In [2]:
import Augmentor

In [8]:
data_cat = Augmentor.Pipeline(r"C:\Users\HOME\Desktop\Hailabs Assignments\dataset\cats")

data_cat.zoom(probability=0.3 , min_factor=0.8 , max_factor=1.5)
data_cat.flip_top_bottom(probability=0.4)
data_cat.random_brightness(probability=0.3 , min_factor=0.3 , max_factor=1.2)
data_cat.random_distortion(probability=1 , grid_width = 4 , grid_height=4 ,magnitude = 8)
data_cat.rotate(probability=0.3, max_left_rotation=10, max_right_rotation=10)
data_cat.flip_left_right(probability=0.3)
data_cat.flip_random(probability=0.3)
data_cat.crop_random(probability=.1, percentage_area=0.3)
data_cat.resize(probability=1, width=100, height=100)
data_cat.random_color(probability=0.5, min_factor=0.4, max_factor=0.9)
data_cat.shear(probability=0.3, max_shear_left=0.2, max_shear_right=0.2)
data_cat.rotate(probability=0.3, max_left_rotation=0.2, max_right_rotation=0.2)

data_cat.sample(1000)

Executing Pipeline:   0%|                                                               | 0/1000 [00:00<?, ? Samples/s]

Initialised with 10 image(s) found.
Output directory set to C:\Users\HOME\Desktop\Hailabs Assignments\dataset\cats\output.

Processing <PIL.Image.Image image mode=RGB size=100x100 at 0x1EFFDC94E20>: 100%|█| 1000/1000 [00:08<00:00, 122.47 Sampl


In [9]:

data_dog = Augmentor.Pipeline(r"C:\Users\HOME\Desktop\Hailabs Assignments\dataset\dogs")

data_dog.zoom(probability=0.3 , min_factor=0.8 , max_factor=1.5)
data_dog.flip_top_bottom(probability=0.4)
data_dog.random_brightness(probability=0.3 , min_factor=0.3 , max_factor=1.2)
data_dog.random_distortion(probability=1 , grid_width = 4 , grid_height=4 ,magnitude = 8)
data_dog.rotate(probability=0.3, max_left_rotation=10, max_right_rotation=10)
data_dog.flip_left_right(probability=0.3)
data_dog.flip_random(probability=0.3)
data_dog.crop_random(probability=.1, percentage_area=0.3)
data_dog.resize(probability=1, width=100, height=100)
data_dog.random_color(probability=0.5, min_factor=0.4, max_factor=0.9)
data_dog.shear(probability=0.3, max_shear_left=0.2, max_shear_right=0.2)
data_dog.rotate(probability=0.3, max_left_rotation=0.2, max_right_rotation=0.2)

data_dog.sample(1000)



Executing Pipeline:   0%|                                                               | 0/1000 [00:00<?, ? Samples/s]

Initialised with 10 image(s) found.
Output directory set to C:\Users\HOME\Desktop\Hailabs Assignments\dataset\dogs\output.

Processing <PIL.Image.Image image mode=RGB size=100x100 at 0x1EFFDC020A0>: 100%|█| 1000/1000 [00:09<00:00, 105.21 Sampl


In [10]:

data_rabbit = Augmentor.Pipeline(r"C:\Users\HOME\Desktop\Hailabs Assignments\dataset\rabbit")

data_rabbit.zoom(probability=0.3 , min_factor=0.8 , max_factor=1.5)
data_rabbit.flip_top_bottom(probability=0.4)
data_rabbit.random_brightness(probability=0.3 , min_factor=0.3 , max_factor=1.2)
data_rabbit.random_distortion(probability=1 , grid_width = 4 , grid_height=4 ,magnitude = 8)
data_rabbit.rotate(probability=0.3, max_left_rotation=10, max_right_rotation=10)
data_rabbit.flip_left_right(probability=0.3)
data_rabbit.flip_random(probability=0.3)
data_rabbit.crop_random(probability=.1, percentage_area=0.3)
data_rabbit.resize(probability=1, width=100, height=100)
data_rabbit.random_color(probability=0.5, min_factor=0.4, max_factor=0.9)
data_rabbit.shear(probability=0.3, max_shear_left=0.2, max_shear_right=0.2)
data_rabbit.rotate(probability=0.3, max_left_rotation=0.2, max_right_rotation=0.2)

data_rabbit.sample(1000)



Processing <PIL.Image.Image image mode=RGB size=100x100 at 0x1EFFD891D00>:   2%| | 15/1000 [00:00<00:17, 56.07 Samples/

Initialised with 10 image(s) found.
Output directory set to C:\Users\HOME\Desktop\Hailabs Assignments\dataset\rabbit\output.

Processing <PIL.Image.Image image mode=RGB size=100x100 at 0x1EFFDBC6D30>: 100%|█| 1000/1000 [00:05<00:00, 181.95 Sampl
